# Instacart Market Basket Analysis

 Instacart, a grocery ordering and delivery app, aims to make it easy to fill your refrigerator and pantry with your personal favorites and staples when you need them. After selecting products through the Instacart app, personal shoppers review your order and do the in-store shopping and delivery for you.

Instacart’s data science team plays a big part in providing this delightful shopping experience. Currently they use transactional data to develop models that predict which products a user will buy again, try for the first time, or add to their cart next during a session. Recently, Instacart open sourced this data - see their blog post on 3 Million Instacart Orders, Open Sourced.

**Task:** Using the anonymized data on customer orders over time to predict which previously purchased products will be in a user’s next order.

![Alt text](image.png)

### Dataset Description

- The dataset is a relational set of files describing customers' orders over time. 
- The goal is to predict which products will be in a user's next order. 
- The dataset is anonymized and contains a sample of over 3 million grocery orders from more than 200,000 Instacart users. 
- For each user, between 4 and 100 of the order per user is provided, with the sequence of products purchased in each order. 
- The week and hour of day the order is also placed, and a relative measure of time between orders. .

**File descriptions:**

- Each entity (customer, product, order, aisle, etc.) has an associated unique id. 
- Most of the files and variable names should be self-explanatory.
- Following are the included datasets:
  - aisles.csv
  - departments.csv
  - order_products__*.csv
  - orders.csv
  - products.csv
  - sample_submission.csv


**Data availibility:** At kaggle https://www.kaggle.com/competitions/instacart-market-basket-analysis/data

In [13]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [26]:
# Define the folder path
folder_path = 'instacart-market-basket-analysis/'

# Load aisles.csv
aisles_df = pd.read_csv(folder_path + 'aisles.csv')

# Load departments.csv
departments_df = pd.read_csv(folder_path + 'departments.csv')

# Load order_products.csv
order_products_df = pd.read_csv(folder_path + 'order_products.csv')

# Load orders.csv
orders_df = pd.read_csv(folder_path + 'orders.csv')

# Load products.csv
products_df = pd.read_csv(folder_path + 'products.csv')

In [27]:
aisles_df.head()

,aisle_id,aisle
0,1,prepared soups salads
1,2,specialty cheeses
2,3,energy granola bars
3,4,instant foods
4,5,marinades meat preparation


In [28]:
departments_df.head()

,department_id,department
0,1,frozen
1,2,other
2,3,bakery
3,4,produce
4,5,alcohol


In [29]:
order_products_df.head()

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


In [31]:
orders_df.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0


In [32]:
products_df.head()

,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13


### Data schema

#### Star Schema

<img src="SALES_FACT-2.png" alt="Types of data" style="max-width: 900px;"/>

**Question-1** Query to calculate the total number of products ordered per department

<pre>
    <code>
        SELECT
            d.department,
            COUNT(*) AS total_products_ordered
        FROM
            fact_order_products fop
        JOIN
            dim_departments d ON fop.department_id = d.department_id
        GROUP BY
            d.department;
    </code>
</pre>

<img src="SQL-Plots/1.png" alt="Types of data" style="max-width: 900px;"/>

**Question-2** Query to find the top 5 aisles with the highest number of reordered products:

<pre>
    <code>
        SELECT
            a.aisle,
            COUNT(*) AS total_reordered
        FROM
            fact_order_products fop
        JOIN
            dim_aisles a ON fop.aisle_id = a.aisle_id
        WHERE
            fop.reordered = TRUE
        GROUP BY
            a.aisle
        ORDER BY
            total_reordered DESC
        LIMIT 5;
    </code>
</pre>

<img src="SQL-Plots/2.png" alt="Types of data" style="max-width: 900px;"/>

**Question-** Query to calculate the average number of products added to the cart per order by day of the week:

<pre>
    <code>
        SELECT
            o.order_dow,
            AVG(fop.add_to_cart_order) AS avg_products_per_order
        FROM
            fact_order_products fop
        JOIN
            dim_orders o ON fop.order_id = o.order_id
        GROUP BY
             o.order_dow;
    </code>
</pre>

<img src="SQL-Plots/3.png" alt="Types of data" style="max-width: 900px;"/>

**Question-** Query to identify the top 10 users with the highest number of unique products ordered:

<pre>
    <code>
        SELECT
            u.user_id,
             COUNT(DISTINCT fop.product_id) AS unique_products_ordered
        FROM
            fact_order_products fop
        JOIN
            dim_users u ON fop.user_id = u.user_id
        GROUP BY
            u.user_id
        ORDER BY
            unique_products_ordered DESC
            LIMIT 10;
    </code>
</pre>

<img src="SQL-Plots/4.png" alt="Types of data" style="max-width: 900px;"/>

### Complete SQL code

<pre>
    <code>
-- Providing access to the AWS s3 bucket where data is stored.
CREATE STAGE my_stage
URL = "s3://dw-snoflake-course-arun/instamart/"
CREDENTIALS = (AWS_Key_id = '' AWS_SECRET_KEY = '');

-- setting the file format that would be loaded from the s3 bucket.
CREATE OR REPLACE FILE FORMAT csv_file_format
TYPE = 'CSV'
FIELD_DELIMITER = ','
SKIP_HEADER = 1
FIELD_OPTIONALLY_ENCLOSED_BY = '"';



-- copying the data from the .csv files in s3 bucket
CREATE TABLE aisles (
        aisle_id INTEGER PRIMARY KEY,
        aisle VARCHAR
    );
    
COPY INTO aisles (aisle_id, aisle)
FROM @my_stage/aisles.csv
FILE_FORMAT = (FORMAT_NAME = 'csv_file_format');


CREATE TABLE departments (
        department_id INTEGER PRIMARY KEY,
        department VARCHAR
    );

COPY INTO departments (department_id, department)
FROM @my_stage/departments.csv
FILE_FORMAT = (FORMAT_NAME = 'csv_file_format');


CREATE OR REPLACE TABLE products (
        product_id INTEGER PRIMARY KEY,
        product_name VARCHAR,
        aisle_id INTEGER,
        department_id INTEGER
    );

COPY INTO products (product_id, product_name, aisle_id, department_id)
FROM @my_stage/products.csv
FILE_FORMAT = (FORMAT_NAME = 'csv_file_format');


CREATE OR REPLACE TABLE orders (
        order_id INTEGER PRIMARY KEY,
        user_id INTEGER,
        eval_set STRING,
        order_number INTEGER,
        order_dow INTEGER,
        order_hour_of_day INTEGER,
        days_since_prior_order INTEGER
    );

COPY INTO orders (order_id, user_id, eval_set, order_number, order_dow, order_hour_of_day, days_since_prior_order)
FROM @my_stage/orders.csv
FILE_FORMAT = (FORMAT_NAME = 'csv_file_format');



CREATE OR REPLACE TABLE order_products (
        order_id INTEGER,
        product_id INTEGER,
        add_to_cart_order INTEGER,
        reordered INTEGER,
        PRIMARY KEY (order_id, product_id)
    );
    
COPY INTO order_products (order_id, product_id, add_to_cart_order, reordered)
FROM @my_stage/order_products.csv
FILE_FORMAT = (FORMAT_NAME = 'csv_file_format');



-- Creating the star schema
-- Creating the dim table

CREATE OR REPLACE TABLE dim_users AS (
  SELECT
    user_id
  FROM
    orders
);

CREATE OR REPLACE TABLE dim_products AS (
  SELECT
    product_id,
    product_name
  FROM
    products
);


CREATE OR REPLACE TABLE dim_aisles AS (
  SELECT
    aisle_id,
    aisle
  FROM
    aisles
);

CREATE OR REPLACE TABLE dim_departments AS (
  SELECT
    department_id,
    department
  FROM
    departments
);

CREATE OR REPLACE TABLE dim_orders AS (
  SELECT
    order_id,
    order_number,
    order_dow,
    order_hour_of_day,
    days_since_prior_order
  FROM
    orders
);

-- now creating the fact table

CREATE TABLE fact_order_products AS (
  SELECT
    op.order_id,
    op.product_id,
    o.user_id,
    p.department_id,
    p.aisle_id,
    op.add_to_cart_order,
    op.reordered
  FROM
    order_products op
  JOIN
    orders o ON op.order_id = o.order_id
  JOIN
    products p ON op.product_id = p.product_id
);

-- --- --- --- -- --- -- --- --- --- --- --- --- --- --- --- --- 
--                Let's do some analytics                     --
-- --- --- --- -- --- -- --- --- --- --- --- --- --- --- --- --- 
-- Query to calculate the total number of products ordered per department:
SELECT
  d.department,
  COUNT(*) AS total_products_ordered
FROM
  fact_order_products fop
JOIN
  dim_departments d ON fop.department_id = d.department_id
GROUP BY
  d.department;

-- Query to find the top 5 aisles with the highest number of reordered products:
SELECT
  a.aisle,
  COUNT(*) AS total_reordered
FROM
  fact_order_products fop
JOIN
  dim_aisles a ON fop.aisle_id = a.aisle_id
WHERE
  fop.reordered = TRUE
GROUP BY
  a.aisle
ORDER BY
  total_reordered DESC
LIMIT 5;

-- Query to calculate the average number of products added to the cart per order by day of the week:
SELECT
  o.order_dow,
  AVG(fop.add_to_cart_order) AS avg_products_per_order
FROM
  fact_order_products fop
JOIN
  dim_orders o ON fop.order_id = o.order_id
GROUP BY
  o.order_dow;

-- Query to identify the top 10 users with the highest number of unique products ordered:
SELECT
  u.user_id,
  COUNT(DISTINCT fop.product_id) AS unique_products_ordered
FROM
  fact_order_products fop
JOIN
  dim_users u ON fop.user_id = u.user_id
GROUP BY
  u.user_id
ORDER BY
  unique_products_ordered DESC
LIMIT 10;

    </code>
</pre>










**Question-**